In [ ]:
import pandas as pd
import numpy as np
import os
import random
import pathlib
import joblib
import glob
import random
from imutils import rotate as rotate
from itertools import product
import PIL
from PIL import Image
import gc

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib.patches as patches
from matplotlib.patches import Polygon


from sklearn.utils import shuffle
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import RandomFlip
from tensorflow.keras.layers import RandomZoom
from tensorflow.keras.layers import RandomRotation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import cv2 as cv
import skimage.io as skio

# Required to read the data from Kaggle
from google.colab import drive
drive.mount('/content/gdrive')
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle"

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [ ]:
!kaggle datasets download -d andrewmvd/isic-2019 --force

In [ ]:

!unzip \*.zip  && rm *.zip

In [ ]:
!wc -l ISIC_2019_Training_GroundTruth.csv
!rm -rf /content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/MEL
!rm -rf /content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/NV
!rm -rf /content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/BCC
!rm -rf /content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/UNK
!rm -rf /content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/AK
!rm -rf /content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/BKL
!rm -rf /content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/DF
!rm -rf /content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/VASC
!rm -rf /content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/SCC

In [ ]:
orig_label_file = 'ISIC_2019_Training_GroundTruth.csv'
orig_label = pd.read_csv(orig_label_file)
print("Number of rows ", len(orig_label.index))
orig_label = shuffle(orig_label)
print("Number of rows ", len(orig_label.index))

split = (0.8, 0.1, 0.1)
split_indices = np.multiply(len(orig_label), split).astype(int)
for idx, val in enumerate(split_indices):
    print(idx, val)
    if idx > 0:
        split_indices[idx] = sum(split_indices[idx - 1 : idx]) + split_indices[idx]
print("Split indecs : ", split_indices)


train_label = orig_label[: split_indices[0]]
test_label = orig_label[split_indices[0] : split_indices[1]]
val_test = orig_label[split_indices[1]:]

print("Train Label Size: ",train_label.size)

In [ ]:
val_test

In [ ]:
!ls /content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/Train/MEL

In [ ]:
import shutil
var_list =['MEL', 'NV','BCC','AK','BKL','DF','VASC','SCC','UNK']
labels = train_label.melt(id_vars =['image'], value_vars =['MEL', 'NV','BCC','AK','BKL','DF','VASC','SCC','UNK'])
labels = labels.loc[labels['value'] == 1.0]
labels= labels.drop(labels.columns[2],axis=1)

base_path = '/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/'
train_base_path = '/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/Train/'
isExist = os.path.exists(train_base_path)
if not isExist:
  os.mkdir(train_base_path)  

for value in var_list:
  path = train_base_path+value
  isExist = os.path.exists(path)
  if not isExist:
    os.mkdir(path)
    print("The new directory is created!")

labels
for index, row in labels.iterrows():
  source = base_path+row['image']+'.jpg';
  dest = train_base_path+row['variable']+'/'+row['image']+'.jpg'
  im = Image.open(source)
  width, height = im.size
  newsize = (96, 96)
  im1 = im.resize(newsize)
  im1.save(source)
  shutil.move(source,dest)
  print(source,dest)
    

   


Moving test images to Test directory

In [ ]:
!ls /content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/Train/MEL

In [ ]:
var_list =['MEL', 'NV','BCC','AK','BKL','DF','VASC','SCC','UNK']
test_labels = test_label.melt(id_vars =['image'], value_vars =['MEL', 'NV','BCC','AK','BKL','DF','VASC','SCC','UNK'])
test_labels = test_labels.loc[test_labels['value'] == 1.0]
test_labels= test_labels.drop(test_labels.columns[2],axis=1)
#test_label
test_base_path = '/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/'
isExist = os.path.exists(test_base_path+'Test/')
if not isExist:
    os.mkdir(test_base_path+'Test/')
    print("The new directory is created!")

test_ext_path = '/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/Test/'

mode = 0o777
for value in var_list:
  test_path = test_ext_path+value
  print(test_path)
  isExist = os.path.exists(test_path)
  if not isExist:
    os.mkdir(test_path)
    print("The new directory is created!")



for index, row in test_labels.iterrows():
  source = test_base_path+row['image']+'.jpg'
  dest = test_ext_path+row['variable']+'/'+row['image']+'.jpg'
  im = Image.open(source)
  width, height = im.size
  newsize = (96, 96)
  im1 = im.resize(newsize)
  im1.save(source)
  shutil.move(source,dest)

In [ ]:
! ls -ltr /content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/Test/MEL/

In [ ]:
data_dir_train = pathlib.Path("/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/Train/")
data_dir_test = pathlib.Path("/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/Test/")


In [ ]:
image_count_train = len(list(data_dir_train.glob('*/*.jpg')))
print(image_count_train)

In [ ]:
image_dataset = tf.keras.preprocessing.image_dataset_from_directory(data_dir_train,batch_size=32,image_size=(96,96),
                                                                    label_mode='categorical',seed=123)

#all the classes of Skin Cancer
class_names = image_dataset.class_names

#Dictionary to store the path of image as per the class
files_path_dict = {}

for c in class_names:
    files_path_dict[c] = list(map(lambda x:str(data_dir_train)+'/'+c+'/'+x,os.listdir(str(data_dir_train)+'/'+c)))
    #print(c)
    
    
#Visualize image 
plt.figure(figsize=(15,15))
index = 0
for c in class_names:
    if (c!='UNK' and c!='Test'):
      path_list = files_path_dict[c][:1]
      index += 1
      #print(path_list)
      plt.subplot(4,2,index)
      plt.imshow(load_img(path_list[0],target_size=(96,96)))
      plt.title(c)
      plt.axis("off")

In [ ]:
def class_distribution_count(directory):
    
    #count number of image in each classes
    count= []
    for path in pathlib.Path(directory).iterdir():
        if path.is_dir():
            count.append(len([name for name in os.listdir(path)
                               if os.path.isfile(os.path.join(path, name))]))
    
    #name of the classes
    sub_directory = [name for name in os.listdir(directory)
                    if os.path.isdir(os.path.join(directory, name))]
    
    #return dataframe with image count and class.
    return pd.DataFrame(list(zip(sub_directory,count)),columns =['Class', 'No. of Image'])

df = class_distribution_count(data_dir_train)
df

In [ ]:
import seaborn as sns
plt.figure(figsize=(10, 8))
sns.barplot(x="No. of Image", y="Class", data=df,
            label="Class")

In [ ]:
!pip install Augmentor

In [ ]:
path_to_training_dataset="/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/Train/"
import Augmentor
for i in class_names:
    if (i!='UNK'):
      p = Augmentor.Pipeline(path_to_training_dataset + i)
      p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
      p.sample(300)  #Adding 300 samples per class to make sure that none of the classes are sparse

In [ ]:
image_count_train = len(list(data_dir_train.glob('*/output/*.jpg')))
print(image_count_train)

In [ ]:
# train dataset 
train_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir_train, batch_size=32,
                                                               image_size=(96,96), label_mode='categorical',
                                                               seed=123,subset="training",
                                                               validation_split=0.2)

#label_mode is categorial, the labels are a float32 tensor of shape (batch_size, num_classes),
#representing a one-hot encoding of the class index.

In [ ]:
val_ds =tf.keras.preprocessing.image_dataset_from_directory(data_dir_train,batch_size=32,
                                                            image_size=(96,96), label_mode='categorical',
                                                            seed=123,subset="validation",
                                                            validation_split=0.2)

In [ ]:
#tf.data.experimental.AUTOTUNE defines appropriate number of processes that are free for working.

#`Dataset.cache()` keeps the images in memory after they're loaded off disk during the first epoch.

#`Dataset.prefetch()` overlaps data preprocessing and model execution while training.


AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
#CNN Model Architecture

#Sequential allows you to create models layer-by-layer  
model = Sequential()

model.add(layers.experimental.preprocessing.Rescaling(1./255,input_shape=(96,96,3)))   #Rescaling Layer

#First Convulation layer
model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

#Second Convulation Layer
model.add(layers.Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

#Third Convulation Layer
model.add(layers.Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

#Dropout layer with 50% Fraction of the input units to drop.
model.add(layers.Dropout(0.5))

#Flatten Layer
##Keras.layers.flatten function flattens the multi-dimensional input tensors into a single dimension.
model.add(layers.Flatten())

#Dense Layer
model.add(layers.Dense(128,activation='relu'))

#Dropout layer with 25% Fraction of the input units to drop.
model.add(layers.Dropout(0.25))

#Dense Layer with softmax activation function.
#Softmax is an activation function that scales numbers/logits into probabilities.
model.add(layers.Dense(len(class_names),activation='softmax'))

model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
#Compile the Model

#Adam optimization: is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
#categorical_crossentropy: Used as a loss function for multi-class classification model where there are two or more output labels.

model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])

#ModelCheckpoint callback is used in conjunction with training using model.fit() to save a model or weights (in a checkpoint file) at some interval,
#so the model or weights can be loaded later to continue the training from the state saved.
checkpoint = ModelCheckpoint("model.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)

#Stop training when a monitored metric has stopped improving.
earlystop = EarlyStopping(monitor="val_accuracy",patience=5,mode="auto",verbose=1)

In [ ]:
epochs = 20
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs,callbacks=[checkpoint,earlystop])

In [ ]:
epochs_range = range(earlystop.stopped_epoch+1)

plt.figure(figsize=(15, 10))
plt.subplot(1, 2, 1)

#Plot Model Accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel(epochs_range)
plt.legend(['train', 'val'], loc='upper left')

#Plot Model Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel(epochs_range)
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
test_labels = test_labels.loc[test_labels['image'] == 'ISIC_0033761']
test_labels

In [ ]:
from glob import glob
Test_image_path = os.path.join(test_ext_path,class_names[-1], '*')
Test_image = glob(Test_image_path)
print(Test_image[0])
Test_image = load_img(Test_image[0],target_size=(96,96,3))
plt.imshow(Test_image)
plt.grid(False)

img = np.expand_dims(Test_image,axis=0)
pred = model.predict(img)
pred = np.argmax(pred)
pred_class = class_names[pred]
print("Actual Class "+ class_names[-1] +'\n'+ "Predictive Class "+pred_class )

**Checking the prediction for a single class VASC**

In [ ]:
tf.random.set_seed(1234)
np.random.seed(1234)
from glob import glob
for c in class_names:
  if (c=='VASC'):
    Test_image_path = os.path.join(test_ext_path,c, '*')
    Test_image = glob(Test_image_path)
    total_count = 0
    match_count = 0
    for tim in Test_image:
      t_image = load_img(tim,target_size=(96,96,3))
      img = np.expand_dims(t_image,axis=0)
      pred = model.predict(img)
      pred = np.argmax(pred)
      pred_class = class_names[pred]
      total_count=total_count+1
      if c==pred_class:
        match_count=match_count+1
print(" Total test count :",total_count)
print("Correct match_count:",match_count)




In [ ]:
data_dir_train

In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

**VGG16 Model**

In [ ]:
#CNN VGG16 Model 

#Sequential allows you to create models layer-by-layer  
model_vgg16 = Sequential()

model_vgg16.add(layers.experimental.preprocessing.Rescaling(1./255,input_shape=(96,96,3)))   #Rescaling Layer

#First Convulation layer
model_vgg16.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu'))
model_vgg16.add(layers.MaxPool2D(pool_size=(2,2)))

#Second Convulation Layer
model_vgg16.add(layers.Conv2D(64,kernel_size=(3,3),activation='relu'))
model_vgg16.add(layers.MaxPool2D(pool_size=(2,2)))

#Third Convulation Layer
model_vgg16.add(layers.Conv2D(128,kernel_size=(3,3),activation='relu'))
model_vgg16.add(layers.MaxPool2D(pool_size=(2,2)))

#Dropout layer with 50% Fraction of the input units to drop.
model_vgg16.add(layers.Dropout(0.5))

#Flatten Layer
##Keras.layers.flatten function flattens the multi-dimensional input tensors into a single dimension.
model_vgg16.add(layers.Flatten())

#Dense Layer
model_vgg16.add(layers.Dense(128,activation='relu'))

#Dropout layer with 25% Fraction of the input units to drop.
model_vgg16.add(layers.Dropout(0.25))

#Dense Layer with softmax activation function.
#Softmax is an activation function that scales numbers/logits into probabilities.
model_vgg16.add(layers.Dense(len(class_names),activation='softmax'))

model_vgg16.summary()


In [ ]:
from tensorflow.keras.utils import plot_model
model_vgg16.build()
plot_model(model_vgg16, to_file='model_plot_vgg16.png', show_shapes=True, show_layer_names=True)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model_vgg16.compile(optimizer=opt,loss="categorical_crossentropy",metrics=["accuracy"])
vgg16_checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='auto')

In [ ]:
# train dataset 
train_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir_train, batch_size=32,
                                                               image_size=(96,96), label_mode='categorical',
                                                               seed=123,subset="training",
                                                               validation_split=0.2)


#label_mode is categorial, the labels are a float32 tensor of shape (batch_size, num_classes),
#representing a one-hot encoding of the class index.

In [ ]:
val_ds =tf.keras.preprocessing.image_dataset_from_directory(data_dir_train,batch_size=32,
                                                            image_size=(96,96), label_mode='categorical',
                                                            seed=123,subset="validation",
                                                            validation_split=0.2)

In [ ]:
print(train_ds.file_paths)

In [ ]:


#print('val_ds @@@@',val_ds)
#history = model1.fit(train_ds, validation_data=val_ds, epochs=epochs,callbacks=[checkpoint,earlystop])
#print(train_ds.shape)
#hist = model1.fit(steps_per_epoch=20,generator=train_ds, validation_data= val_ds, validation_steps=10,epochs=20,callbacks=[checkpoint,earlystop])
hist = model_vgg16.fit(train_ds, validation_data= val_ds,epochs=20,callbacks=[vgg16_checkpoint,early])

In [ ]:
epochs_range = range(earlystop.stopped_epoch+1)

plt.figure(figsize=(15, 10))
plt.subplot(1, 2, 1)

#Plot Model Accuracy
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel(epochs_range)
plt.legend(['train', 'val'], loc='upper left')

#Plot Model Loss
plt.subplot(1, 2, 2)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel(epochs_range)
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
tf.random.set_seed(1234)
np.random.seed(1234)
from glob import glob
for c in class_names:
  if (c=='BCC'):
    Test_image_path = os.path.join(test_ext_path,c, '*')
    Test_image = glob(Test_image_path)
    total_count = 0
    match_count = 0
    for tim in Test_image:
      t_image = load_img(tim,target_size=(96,96,3))
      img = np.expand_dims(t_image,axis=0)
      pred = model_vgg16.predict(img)
      pred = np.argmax(pred)
      pred_class = class_names[pred]
      print("Prediction ",pred_class," Class ",c," Tim ",tim)

      total_count=total_count+1
      if c==pred_class:
        match_count=match_count+1
print(" Total test count :",total_count)
print("Correct match_count:",match_count)

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir_test,
    label_mode="categorical",
    batch_size=32,
    image_size=(96, 96),
    seed=123,
    validation_split=None,
    subset=None
)

In [ ]:
test_results = model_vgg16.evaluate(test_ds)
print('\nTest Acc. {:.2f}%'.format(test_results[1]*100))